In [1]:
#SciKit DSC540 HW4
'''created by Casey Bennett 2018, www.CaseyBennett.com'''

import sys
import csv
import math
import numpy as np
from operator import itemgetter
import time
import copy

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.externals import joblib
from sklearn.feature_selection import RFE, VarianceThreshold, SelectFromModel
from sklearn.feature_selection import SelectKBest, mutual_info_regression, mutual_info_classif, chi2
from sklearn import metrics
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import KBinsDiscretizer, scale

#Handle annoying warnings
import warnings, sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.ConvergenceWarning)




In [2]:
#############################################################################
#
# Global parameters
#
#####################

target_idx=0                                        #Index of Target variable
cross_val=1                                         #Control Switch for CV                                                                                                                                                                                           
norm_target=0                                       #Normalize target switch
norm_features=1                                     #Normalize target switch                                                                                                    
binning=0                                           #Control Switch for Bin Target
bin_cnt=2                                           #If bin target, this sets number of classes
feat_select=1                                       #Control Switch for Feature Selection                                                                                        
fs_type=4                                           #Feature Selection type (1=Stepwise Backwards Removal, 2=Wrapper Select, 3=Univariate Selection)
lv_filter=0                                         #Control switch for low variance filter on features
feat_start=1                                        #Start column of features
k_cnt=5                                             #Number of 'Top k' best ranked features to select, only applies for fs_types 1 and 3

#Set global model parameters
rand_st=1                                           #Set Random State variable for randomizing splits on runs




In [3]:
#############################################################################
#
# Wrapper Feat Select Helper
#
##########################################

#Recursive Function for searching thru feature space
def feat_space_search(arr, curr_idx):
    '''Setup currently as exhuastive search, but could be changed to use
       greedy search, random search, genetic algorithms, etc. ... also
       no regularization, so probably selects more features than necessary'''
    global roll_idx, combo_ctr, best_score, sel_idx
    
    if curr_idx==feat_cnt:
        #If end of feature array, roll thru combinations
        roll_idx=roll_idx+1
        print ("Combos Searched so far:", combo_ctr, "Current Best Score:", best_score)
        for i in range(roll_idx, len(arr)):
            arr[i]=0
        if roll_idx<feat_cnt-1:
            feat_space_search(arr, roll_idx+1)                                                                      #Recurse till end of rolls
        
    else:
        #Else setup next feature combination and calc performance
        arr[curr_idx]=1
        data=data_np#_wrap                                                                                          #Temp array to hold data
        temp_del=[i for i in range(len(arr)) if arr[i]==0]                                                          #Pick out features not in this combo, and remove
        data = np.delete(data, temp_del, axis=1)
        data_train, data_test, target_train, target_test = train_test_split(data, target_np, test_size=0.35)                

        if binning==1:
            if bin_cnt<=2:
                scorers = {'Accuracy': 'accuracy', 'roc_auc': 'roc_auc'}
                scores = cross_validate(clf, data_np, target_np, scoring=scorers, cv=5) 
                score = scores['test_roc_auc'].mean()                                                               #AUC
            else:
                sscorers = {'Accuracy': 'accuracy'}
                scores = cross_validate(clf, data_np, target_np, scoring=scorers, cv=5) 
                score = scores['test_Accuracy'].mean()                                                              #Accuracy
            print('Random Forest Acc/AUC:', curr_idx, feat_arr, len(data[0]), score)
            if score>best_score:                                                                                    #Compare performance and update sel_idx and best_score, if needed
                best_score=score
                sel_idx=copy.deepcopy(arr) 
                
        if binning==0:
            scorers = {'Neg_MSE': 'neg_mean_squared_error', 'expl_var': 'explained_variance'}
            scores = cross_validate(rgr, data, target_np, scoring=scorers, cv=5)    
            score = np.asarray([math.sqrt(-x) for x in scores['test_Neg_MSE']]).mean()                              #RMSE
            print('Random Forest RMSE:', curr_idx, feat_arr, len(data[0]), score)
            if score<best_score:                                                                                    #Compare performance and update sel_idx and best_score, if needed
                best_score=score
                sel_idx=copy.deepcopy(arr) 

        #move to next feature index and recurse
        combo_ctr+=1  
        curr_idx+=1
        feat_space_search(arr, curr_idx)                                                                            #Recurse till end of iteration for roll




In [4]:
#############################################################################
#
# Load Data
#
#####################

file1= csv.reader(open('WineQuality_Red.csv'), delimiter=',', quotechar='"')

#Read Header Line
header=next(file1)            

#Read data
data=[]
target=[]
for row in file1:
    #Load Target
    if row[target_idx]=='':                         #If target is blank, skip row                       
        continue
    else:
        target.append(float(row[target_idx]))       #If pre-binned class, change float to int

    #Load row into temp array, cast columns  
    temp=[]
                 
    for j in range(feat_start,len(header)):
        if row[j]=='':
            temp.append(float())
        else:
            temp.append(float(row[j]))

    #Load temp into Data array
    data.append(temp)
  
#Test Print
print(header)
print(len(target),len(data))
print('\n')

data_np=np.asarray(data)
target_np=np.asarray(target)




['Class', 'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']
1599 1599




In [5]:
#############################################################################
#
# Preprocess data
#
##########################################

if norm_target==1:
    #Target normalization for continuous values
    target_np=scale(target_np)

if norm_features==1:
    #Feature normalization for continuous values
    data_np=scale(data_np)

if binning==1:
    #Discretize Target variable with KBinsDiscretizer
    enc = KBinsDiscretizer(n_bins=[bin_cnt], encode='ordinal', strategy='quantile')                         #Strategy here is important, quantile creating equal bins, but kmeans prob being more valid "clusters"
    target_np_bin = enc.fit_transform(target_np.reshape(-1,1))

    #Get Bin min/max
    temp=[[] for x in range(bin_cnt+1)]
    for i in range(len(target_np)):
        for j in range(bin_cnt):
            if target_np_bin[i]==j:
                temp[j].append(target_np[i])

    for j in range(bin_cnt):
        print('Bin', j, ':', min(temp[j]), max(temp[j]), len(temp[j]))
    print('\n')

    #Convert Target array back to correct shape
    target_np=np.ravel(target_np_bin)
    



In [6]:
#############################################################################
#
# Feature Selection
#
##########################################

#Low Variance Filter
if lv_filter==1:
    print('--LOW VARIANCE FILTER ON--', '\n')
    
    #LV Threshold
    sel = VarianceThreshold(threshold=0.5)                                      #Removes any feature with less than 20% variance
    fit_mod=sel.fit(data_np)
    fitted=sel.transform(data_np)
    sel_idx=fit_mod.get_support()

    #Get lists of selected and non-selected features (names and indexes)
    temp=[]
    temp_idx=[]
    temp_del=[]
    for i in range(len(data_np[0])):
        if sel_idx[i]==1:                                                           #Selected Features get added to temp header
            temp.append(header[i+feat_start])
            temp_idx.append(i)
        else:                                                                       #Indexes of non-selected features get added to delete array
            temp_del.append(i)

    print('Selected', temp)
    print('Features (total, selected):', len(data_np[0]), len(temp))
    print('\n')

    #Filter selected columns from original dataset
    header = header[0:feat_start]
    for field in temp:
        header.append(field)
    data_np = np.delete(data_np, temp_del, axis=1)                                 #Deletes non-selected features by index


#Feature Selection
if feat_select==1:
    '''Three steps:
       1) Run Feature Selection
       2) Get lists of selected and non-selected features
       3) Filter columns from original dataset
       '''
    
    print('--FEATURE SELECTION ON--', '\n')
    
    ##1) Run Feature Selection #######
    if fs_type==1:
        #Stepwise Recursive Backwards Feature removal
        if binning==1:
            clf = RandomForestClassifier(n_estimators=200, max_depth=None, min_samples_split=3, criterion='entropy', random_state=rand_st)
            sel = RFE(clf, n_features_to_select=k_cnt, step=.1)
            print('Stepwise Recursive Backwards - Random Forest: ')
        if binning==0:
            rgr = RandomForestRegressor(n_estimators=500, max_depth=None, min_samples_split=3, criterion='mse', random_state=rand_st)
            sel = RFE(rgr, n_features_to_select=k_cnt, step=.1)
            print('Stepwise Recursive Backwards - Random Forest: ')
            
        fit_mod=sel.fit(data_np, target_np)
        print(sel.ranking_)
        sel_idx=fit_mod.get_support()      

    if fs_type==2:
        #Wrapper Select via model
        if binning==1:
            clf = '''Unused in this homework'''
            sel = SelectFromModel(clf, prefit=False, threshold='mean', max_features=None)                                                           #to select only based on max_features, set to integer value and set threshold=-np.inf
            print ('Wrapper Select: ')
        if binning==0:
            rgr = SVR(kernel='linear', gamma=0.1, C=1.0)
            sel = SelectFromModel(rgr, prefit=False, threshold='mean', max_features=None)
            print ('Wrapper Select: ')
            
        fit_mod=sel.fit(data_np, target_np)    
        sel_idx=fit_mod.get_support()

    if fs_type==3:
        if binning==1:                                                              ######Only work if the Target is binned###########
            #Univariate Feature Selection - Chi-squared
            sel=SelectKBest(chi2, k=k_cnt)
            fit_mod=sel.fit(data_np, target_np)                                         #will throw error if any negative values in features, so turn off feature normalization, or switch to mutual_info_classif
            print ('Univariate Feature Selection - Chi2: ')
            sel_idx=fit_mod.get_support()

        if binning==0:                                                              ######Only work if the Target is continuous###########
            #Univariate Feature Selection - Mutual Info Regression
            sel=SelectKBest(mutual_info_regression, k=k_cnt)
            fit_mod=sel.fit(data_np, target_np)
            print ('Univariate Feature Selection - Mutual Info: ')
            sel_idx=fit_mod.get_support()

        #Print ranked variables out sorted
        temp=[]
        scores=fit_mod.scores_
        for i in range(feat_start, len(header)):            
            temp.append([header[i], float(scores[i-feat_start])])

        print('Ranked Features')
        temp_sort=sorted(temp, key=itemgetter(1), reverse=True)
        for i in range(len(temp_sort)):
            print(i, temp_sort[i][0], ':', temp_sort[i][1])
        print('\n')

    if fs_type==4:
        #Full-blown Wrapper Select (from any kind of ML model)        
        if binning==1:                                                              ######Only work if the Target is binned###########
            start_ts=time.time()
            sel_idx=[]                                                                                      #Empty array to hold optimal selected feature set
            best_score=0                                                                                    #For classification compare Accuracy or AUC, higher is better, so start with 0
            feat_cnt=len(data_np[0])
            #Create Wrapper model
            clf = '''Unused in this homework'''                                 #This could be any kind of classifier model
      
        if binning==0:                                                              ######Only work if the Target is continuous###########
            start_ts=time.time()
            sel_idx=[]                                                                                      #Empty array to hold optimal selected feature set
            best_score=sys.float_info.max                                                                   #For regression compare RMSE, lower is better, so start with max sys float value
            feat_cnt=len(data_np[0])
            #Create Wrapper model
            rgr = SVR(kernel='linear', gamma=0.1, C=1.0)                    #This could be any kind of regressor model         
        
        #Loop thru feature sets
        roll_idx=0
        combo_ctr=0
        feat_arr=[0 for col in range(feat_cnt)]                                         #Initialize feature array
        for idx in range(feat_cnt):
            roll_idx=idx
            feat_space_search(feat_arr, idx)                                           #Recurse
            feat_arr=[0 for col in range(feat_cnt)]                                     #Reset feature array after each iteration
        
        print('# of Feature Combos Tested:', combo_ctr)
        print(best_score, sel_idx, len(data_np[0]))
        print("Wrapper Feat Sel Runtime:", time.time()-start_ts)

    ##2) Get lists of selected and non-selected features (names and indexes) #######
    temp=[]
    temp_idx=[]
    temp_del=[]
    for i in range(len(data_np[0])):
        if sel_idx[i]==1:                                                           #Selected Features get added to temp header
            temp.append(header[i+feat_start])
            temp_idx.append(i)
        else:                                                                       #Indexes of non-selected features get added to delete array
            temp_del.append(i)
    print('Selected', temp)
    print('Features (total/selected):', len(data_np[0]), len(temp))
    print('\n')
            
                
    ##3) Filter selected columns from original dataset #########
    header = header[0:feat_start]
    for field in temp:
        header.append(field)
    data_np = np.delete(data_np, temp_del, axis=1)                                 #Deletes non-selected features by index)
    
    



--FEATURE SELECTION ON-- 

Random Forest RMSE: 0 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 1 0.8838595069304984
Random Forest RMSE: 1 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0] 2 0.7583279027750782
Random Forest RMSE: 2 [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0] 3 0.7601747499953804
Random Forest RMSE: 3 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0] 4 0.7614131615860809
Random Forest RMSE: 4 [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0] 5 0.7529034747655474
Random Forest RMSE: 5 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0] 6 0.753997337443167
Random Forest RMSE: 6 [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0] 7 0.7351744756641166
Random Forest RMSE: 7 [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0] 8 0.7068355888893423
Random Forest RMSE: 8 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0] 9 0.7055446816953973
Random Forest RMSE: 9 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0] 10 0.6890040034487401
Random Forest RMSE: 10 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 11 0.6617118403123001
Combos Searched so far: 11 Current Best Score: 0.6617118403123001
Random Forest RMSE: 2 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0] 2 

Random Forest RMSE: 9 [0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0] 4 0.7557081294835408
Random Forest RMSE: 10 [0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1] 5 0.6780720787067034
Combos Searched so far: 96 Current Best Score: 0.6596825116948701
Random Forest RMSE: 8 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0] 2 0.7537696358599136
Random Forest RMSE: 9 [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0] 3 0.7549904805626544
Random Forest RMSE: 10 [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1] 4 0.676474690563204
Combos Searched so far: 99 Current Best Score: 0.6596825116948701
Random Forest RMSE: 9 [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0] 2 0.7548117487171598
Random Forest RMSE: 10 [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1] 3 0.6765905004226405
Combos Searched so far: 101 Current Best Score: 0.6596825116948701
Random Forest RMSE: 10 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1] 2 0.6765697591042016
Combos Searched so far: 102 Current Best Score: 0.6596825116948701
Random Forest RMSE: 2 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0] 1 0.8186783832393199
Random Forest RMSE: 3 [0, 0, 1, 1, 0, 

Random Forest RMSE: 9 [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0] 4 0.7841692116083575
Random Forest RMSE: 10 [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1] 5 0.6927013049520467
Combos Searched so far: 184 Current Best Score: 0.6596825116948701
Random Forest RMSE: 8 [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0] 2 0.8475574819530334
Random Forest RMSE: 9 [0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0] 3 0.7907309330961789
Random Forest RMSE: 10 [0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1] 4 0.6906594587606218
Combos Searched so far: 187 Current Best Score: 0.6596825116948701
Random Forest RMSE: 9 [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0] 2 0.7915577488358879
Random Forest RMSE: 10 [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1] 3 0.6988928844453153
Combos Searched so far: 189 Current Best Score: 0.6596825116948701
Random Forest RMSE: 10 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1] 2 0.7265796229890172
Combos Searched so far: 190 Current Best Score: 0.6596825116948701
Random Forest RMSE: 5 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0] 1 0.8611320879579424
Random Forest RMSE: 6 [0, 0, 0, 0, 

In [7]:
#############################################################################
#
# Train SciKit Models
#
##########################################

print('--ML Model Output--', '\n')

#Test/Train split
data_train, data_test, target_train, target_test = train_test_split(data_np, target_np, test_size=0.35)

####Regressors####
if binning==0 and cross_val==0:
    #SciKit
    '''Test/Train split unused in this homework, skip down to CV section'''
 

                                                                                                                          

####Cross-Val Regressors####
if binning==0 and cross_val==1:
    #Setup Crossval regression scorers
    scorers = {'Neg_MSE': 'neg_mean_squared_error', 'expl_var': 'explained_variance'} 
    
    #SciKit SVM - Cross Val
    start_ts=time.time()
    rgr=SVR(kernel='linear', gamma=0.1, C=1.0)
    scores=cross_validate(rgr, data_np, target_np, scoring=scorers, cv=5)                                                                                                  

    scores_RMSE = np.asarray([math.sqrt(-x) for x in scores['test_Neg_MSE']])                                       #Turns negative MSE scores into RMSE
    scores_Expl_Var = scores['test_expl_var']
    print("SVM RMSE:: %0.2f (+/- %0.2f)" % ((scores_RMSE.mean()), (scores_RMSE.std() * 2)))
    print("SVM Expl Var: %0.2f (+/- %0.2f)" % ((scores_Expl_Var.mean()), (scores_Expl_Var.std() * 2)))
    print("CV Runtime:", time.time()-start_ts)


    #SciKit LDA - Cross Val
    '''Replace comment HERE''' 





--ML Model Output-- 

SVM RMSE:: 0.66 (+/- 0.02)
SVM Expl Var: 0.30 (+/- 0.18)
CV Runtime: 0.692650318145752
